## Load and transform data

In [1]:
rawData = open("message.txt") do file
    (read(file,String))
end;

In [2]:
function hexToBinary(input::AbstractString)
    myIObuffer = IOBuffer();
    hexBinMap = Dict('0' => "0000",'1' => "0001",'2' => "0010",'3' => "0011",'4' => "0100",'5' => "0101",'6' => "0110",'7' => "0111",'8' => "1000",'9' => "1001",'A' => "1010",'B' => "1011",'C' => "1100",'D' => "1101",'E' => "1110",'F' => "1111")
    
    for c in input
        #println(c," ",hexBinMap[c])
        write(myIObuffer,hexBinMap[c])
    end
    #println(read(myIObuffer,String))
    return read(seekstart(myIObuffer),String)
end

hexToBinary (generic function with 1 method)

In [49]:
dummy="02329345AF00CA"
hexToBinary(dummy)

0 0000
2 0010
3 0011
2 0010
9 1001
3 0011
4 0100
5 0101
A 1010
F 1111
0 0000
0 0000
C 1100
A 1010



"00000010001100101001001101000101101011110000000011001010"

In [3]:
message = hexToBinary(rawData);

## Part 1

In [4]:
function decodePacket(message::AbstractString,sidx::Integer,cache::Array)
    packVersion = parse(Int,SubString(message,sidx,sidx+2),base=2)
    packType = parse(Int,SubString(message,sidx+3,sidx+5),base=2)
    nbitsMessage = 6
    push!(cache,packVersion)
    #print("$(sidx) $(packVersion) $(packType) ")
    
    if packType == 4 #Packet is value
        lastGroup = false
        idx = sidx+6
        valBinary = ""
        while !lastGroup
            if message[idx] == '0' # typeof(message[i]) is char, so test for '0' instead of 0
                lastGroup = true
            end
            valBinary = string(valBinary,message[idx+1:idx+4])
            idx+=5
            nbitsMessage+=5
        end
        value = parse(Int,valBinary,base=2)
        #println("$(valBinary) $(value)")
        
    else #Packet is operator
        lengthType = parse(Int,message[sidx+6])
        nbitsMessage +=1
        
        #Packet contains total length of subpackets in bits
        if lengthType == 0 
            bitsSubPack = parse(Int,SubString(message,sidx+7,sidx+21),base=2) # 15 bits: indx 7 to 7+14 (inclusive)
            nbitsMessage += (bitsSubPack+15)
            #println("$(lengthType) $(bitsSubPack)")
            bitsUsed = 0
            idx = sidx+22
            while bitsUsed < bitsSubPack
                lengthPack = decodePacket(message,idx,cache)
                bitsUsed +=lengthPack
                idx +=lengthPack
            end
        
        #Packet contains number of subpackets
        else
            nSubPacks = parse(Int,SubString(message,sidx+7,sidx+17),base=2) # 11 bits: indx 7 to 7+10 (inclusive)
            nbitsMessage += 11
            #println("$(lengthType) $(nSubPacks)")
            
            newSidx = sidx+18
            for n in 1:nSubPacks
                bitsSubPack = decodePacket(message,newSidx,cache)
                newSidx +=bitsSubPack
                nbitsMessage +=bitsSubPack
            end
        end
    end
    return nbitsMessage
end

function decodeMessage(message::AbstractString)
    eom = length(message)
    idx = 1
    cache = Array{Int}(undef,0)
    sizehint!(cache,100)
    
    while idx<=eom-6
        idx+=decodePacket(message,idx,cache)       
    end
    return cache
end

decodeMessage (generic function with 1 method)

In [22]:
typeof(message)

String

In [24]:
typeof(test0)

String

In [44]:
versNum = decodeMessage(message);

In [45]:
sum(versNum)

934

In [26]:
length(message)

5464

### Testing message decoder

In [5]:
test0 = hexToBinary("38006F45291200")
test1 = hexToBinary("EE00D40C823060")
testA = hexToBinary("8A004A801A8002F478")
testB = hexToBinary("620080001611562C8802118E34")
testC = hexToBinary("C0015000016115A2E0802F182340")
testD = hexToBinary("A0016C880162017C3686B18A3D4780");

In [7]:
decodePacket(test1,1)

1 7 3 1 3
19 2 4 0001 1
30 4 4 0010 2
41 1 4 0011 3


51

In [8]:
length(test1)

56

In [13]:
decodePacket(test0,1)

1 1 6 0 27
23 6 4 1010 10
34 2 4 00010100 20


49

In [14]:
decodePacket(testA,1)

1 4 2 1 1
19 1 2 1 1
37 5 2 0 11
59 6 4 1111 15


69

In [17]:
4+1+5+6

16

In [15]:
decodePacket(testB,1)

1 3 0 1 2
19 0 0 0 22
41 0 4 1010 10
52 5 4 1011 11
63 1 0 1 2
81 0 4 1100 12
92 3 4 1101 13


102

In [18]:
3+0+0+5+1+0+3

12

In [16]:
decodePacket(testC,1)

1 6 0 0 84
23 0 0 0 22
45 0 4 1010 10
56 6 4 1011 11
67 4 0 1 2
85 7 4 1100 12
96 0 4 1101 13


106

In [19]:
6+0+0+6+4+7+0

23

In [20]:
decodePacket(testD,1)

1 5 0 0 91
23 1 0 1 1
41 3 0 1 5
59 7 4 0110 6
70 6 4 0110 6
81 5 4 1100 12
92 2 4 1111 15
103 2 4 1111 15


113

## Part 2

In [25]:
function interpretPackage(message::AbstractString,sidx::Integer)
    packVersion = parse(Int,SubString(message,sidx,sidx+2),base=2)
    packType = parse(Int,SubString(message,sidx+3,sidx+5),base=2)
    nbitsMessage = 6
    #print("$(sidx) $(packVersion) $(packType) ")
    
    result = 0
    
    if packType == 4 #Packet is value
        lastGroup = false
        idx = sidx+6
        valBinary = ""
        while !lastGroup
            if message[idx] == '0' # typeof(message[i]) is char, so test for '0' instead of 0
                lastGroup = true
            end
            valBinary = string(valBinary,message[idx+1:idx+4])
            idx+=5
            nbitsMessage+=5
        end
        result = parse(Int,valBinary,base=2)
        #println("$(valBinary) $(result)")
        
    else #Packet is operator
        lengthType = parse(Int,message[sidx+6])
        nbitsMessage +=1
        
        allResults = Array{Int}(undef,0)
        ## decode package
        #Packet contains total length of subpackets in bits
        if lengthType == 0 
            bitsSubPack = parse(Int,SubString(message,sidx+7,sidx+21),base=2) # 15 bits: indx 7 to 7+14 (inclusive)
            nbitsMessage += (bitsSubPack+15)
            #println("$(lengthType) $(bitsSubPack)")
            bitsUsed = 0
            idx = sidx+22
            while bitsUsed < bitsSubPack
                lengthPack,resultPack = interpretPackage(message,idx)
                bitsUsed +=lengthPack
                idx +=lengthPack
                push!(allResults,resultPack)
            end
        
        #Packet contains number of subpackets
        else
            nSubPacks = parse(Int,SubString(message,sidx+7,sidx+17),base=2) # 11 bits: indx 7 to 7+10 (inclusive)
            nbitsMessage += 11
            #println("$(lengthType) $(nSubPacks)")
            
            newSidx = sidx+18
            for n in 1:nSubPacks
                bitsSubPack,resultPack = interpretPackage(message,newSidx)
                newSidx +=bitsSubPack
                nbitsMessage +=bitsSubPack
                push!(allResults,resultPack)
            end
        end
        
        #interpret package
        
        if packType == 0
            result = sum(allResults)
        elseif packType == 1
            result = prod(allResults)
        elseif packType == 2
            result = minimum(allResults)
        elseif packType == 3
            result = maximum(allResults)
        elseif packType == 5
            result = allResults[1]>allResults[2] ? 1 : 0
        elseif packType == 6
            result = allResults[1]<allResults[2] ? 1 : 0
        else
            result = allResults[1]==allResults[2] ? 1 : 0
        end
    end
    
    return nbitsMessage,result
end

function interpretMessage(message::AbstractString)
    eom = length(message)
    idx = 1
    
    result=0
    while idx<=eom-6
        packSize,result=interpretPackage(message,idx) 
        idx+=packSize
    end
    return result
end

interpretMessage (generic function with 1 method)

In [26]:
interpretMessage(message)

912901337844

### Testing message interpreter

In [17]:
testa = hexToBinary("C200B40A82")
testb = hexToBinary("04005AC33890")
testc = hexToBinary("880086C3E88112")
testd = hexToBinary("CE00C43D881120")
teste = hexToBinary("D8005AC2A8F0")
testf = hexToBinary("F600BC2D8F")
testg = hexToBinary("9C005AC2F8F0")
testh = hexToBinary("9C0141080250320F1802104A08");

In [14]:
interpretMessage(testa)

1 6 0 1 2
19 6 4 0001 1
30 2 4 0010 2


3

In [18]:
interpretMessage(testb)

1 0 1 0 22
23 5 4 0110 6
34 3 4 1001 9


54

In [19]:
interpretMessage(testc)

1 4 2 0 33
23 5 4 0111 7
34 6 4 1000 8
45 0 4 1001 9


7

In [20]:
interpretMessage(testd)

1 6 3 1 3
19 0 4 0111 7
30 5 4 1000 8
41 0 4 1001 9


9

In [21]:
interpretMessage(teste)

1 6 6 0 22
23 5 4 0101 5
34 2 4 1111 15


1

In [22]:
interpretMessage(testf)

1 7 5 1 2
19 7 4 0101 5
30 5 4 1111 15


0

In [23]:
interpretMessage(testg)

1 4 7 0 22
23 5 4 0101 5
34 7 4 1111 15


0

In [24]:
interpretMessage(testh)

1 4 7 0 80
23 2 0 1 2
41 2 4 0001 1
52 4 4 0011 3
63 6 1 1 2
81 0 4 0010 2
92 2 4 0010 2


1